<a href="https://colab.research.google.com/github/NhipCau/FastAutoAudioStrip-Name/blob/main/FastAutoStrip%26Name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#GoogleDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#ライブラリをインストール
!pip install pydub
!pip install openpyxl
!pip install webrtcvad

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73583 sha256=110503ae09cd2422f4949428b2be737b16b7b5b8184f4f30bdd3017edf70b4f1
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee2f
Successfully built webrtcvad


In [4]:
import webrtcvad
import openpyxl
import os
import soundfile as sf
import numpy as np

# ===== 設定 =====
input_audio_path = "/content/drive/MyDrive/Colab Notebooks/BtlMsg_sample.wav"
excel_path = "/content/drive/MyDrive/Colab Notebooks/BtlMsg_Rename_test.xlsx"
output_dir = "/content/drive/MyDrive/Colab Notebooks/output_wavs"

target_sample_rate = 48000
vad_aggressiveness = 2
silence_thresh = -80  # dBFS
min_silence_len_ms = 12000
pre_buffer_sec = 1
post_buffer_sec = 1
bit_depth = 24  # 16, 24, or 32

filename_column_index = 3
filename_start_row = 1

# ===== 出力フォルダ作成 =====
os.makedirs(output_dir, exist_ok=True)

# ===== Excelからファイル名を取得 =====
wb = openpyxl.load_workbook(excel_path)
sheet = wb.active
filenames = [
    row[filename_column_index]
    for row in sheet.iter_rows(min_row=filename_start_row, values_only=True)
    if row[filename_column_index]
]

# ===== 音声読み込み（float32 or int24） =====
audio, sr = sf.read(input_audio_path)
if sr != target_sample_rate:
    raise ValueError(f"Sample rate mismatch: expected {target_sample_rate}, got {sr}")

# ===== webrtcvad用に16bit PCMへ一時変換 =====
audio_int16 = (audio * 32767).astype(np.int16)
raw_audio = audio_int16.tobytes()

frame_duration = 30  # ms
frame_size = int(target_sample_rate * frame_duration / 1000) * 2  # 2 bytes/sample

frames = [
    (i * frame_duration, raw_audio[i * frame_size:(i + 1) * frame_size])
    for i in range(len(raw_audio) // frame_size)
]

# ===== VADで音声区間を検出 =====
vad = webrtcvad.Vad(vad_aggressiveness)
speech_segments = []
is_speech = False
segment_start = 0

for i, (timestamp, frame) in enumerate(frames):
    if len(frame) < frame_size:
        break
    if vad.is_speech(frame, target_sample_rate):
        if not is_speech:
            segment_start = timestamp
            is_speech = True
    else:
        if is_speech:
            segment_end = timestamp
            speech_segments.append((segment_start, segment_end))
            is_speech = False

if is_speech:
    speech_segments.append((segment_start, timestamp + frame_duration))

# ===== dBFSでフィルタリング =====
def rms_dbfs(segment):
    rms = np.sqrt(np.mean(segment**2))
    return 20 * np.log10(rms + 1e-10)

filtered_segments = []
for start_ms, end_ms in speech_segments:
    start = int(start_ms * target_sample_rate // 1000)
    end = int(end_ms * target_sample_rate // 1000)
    segment = audio[start:end]
    if rms_dbfs(segment) >= silence_thresh:
        filtered_segments.append((start_ms, end_ms))

# ===== 最小サイレンス長でマージ =====
merged_segments = []
if filtered_segments:
    current_start, current_end = filtered_segments[0]
    for start_ms, end_ms in filtered_segments[1:]:
        if start_ms - current_end < min_silence_len_ms:
            current_end = end_ms
        else:
            merged_segments.append((current_start, current_end))
            current_start, current_end = start_ms, end_ms
    merged_segments.append((current_start, current_end))

# ===== バッファを加えて保存 =====
pre_buffer = int(pre_buffer_sec * 1000)
post_buffer = int(post_buffer_sec * 1000)

for i, (start_ms, end_ms) in enumerate(merged_segments):
    if i < len(filenames):
        start = max(0, int((start_ms - pre_buffer) * target_sample_rate // 1000))
        end = min(len(audio), int((end_ms + post_buffer) * target_sample_rate // 1000))
        chunk = audio[start:end]

        # ビット深度に応じた subtype を選択
        if bit_depth == 16:
            subtype = 'PCM_16'
        elif bit_depth == 24:
            subtype = 'PCM_24'
        elif bit_depth == 32:
            subtype = 'PCM_32'
        else:
            raise ValueError("Invalid bit depth. Choose 16, 24, or 32.")

        output_path = os.path.join(output_dir, f"{filenames[i]}.wav")
        sf.write(output_path, chunk, samplerate=target_sample_rate, subtype=subtype)
        print(f"保存しました: {output_path}")
    else:
        print(f"ファイル名が不足しています（チャンク{i+1}）")

# ===== バッファを加えて保存 =====
# ...（保存処理のループ）

処理ファイル数 = len(merged_segments)
保存ファイル数 = min(len(merged_segments), len(filenames))

print(f"✅ 処理が完了しました。{処理ファイル数} 件を処理し、{保存ファイル数} 件を保存しました。")


保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200000.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200100.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200200.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200300.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200500.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200600.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200601.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200602.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200603.wav
保存しました: /content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200604.wav
保存しました: /content/dri

In [2]:
#Pythonで特定フォルダのファイル数をカウントして表示

import os

# ファイル数を調べたいフォルダのパス
path = "/content/drive/MyDrive/Colab Notebooks/output_wavs"

# フォルダ内の全ファイル名をリスト化
files = os.listdir(path)

# リストの長さ（ファイル数）を取得
count = len(files)

# ファイル数を確認
print(count)


935


In [7]:
# Pythonでサブフォルダのファイル数を確認
# サブフォルダが存在しない場合はエラー
# 以下のようなツリー構造の全ファイル数をカウントして表示
#images
#   ├─ahukono
#   ├─ahurikawasimimizuku
#   ├─...
#   ├─uraru

import os

path = "/content/drive/MyDrive/Colab Notebooks/output_wavs"

total = 0
for dir_path in os.listdir(path):
    target_dir = path + "/" + dir_path
    files = os.listdir(target_dir)
    count = len(files)
    total = total + count
    print(dir_path, ":", count)
print("file total:", total)


NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/Colab Notebooks/output_wavs/Em100_00_BtlMsgInfoText_MSG_200000.wav'